In [ ]:
%run setup.py build_ext --inplace

In [ ]:
from klrw.combinatorial_ebranes import CombinatorialEBrane

In [ ]:
n = 6 #number of punctures
k = 2 #number of E-branes

I = CombinatorialEBrane(n,k)

I.apply_braid([-4,3,-2,1,-3,-3,-2,-4]) #8_19
#I.apply_braid([2,3,4,4,4,-3,4,4,4,2]) #8_5

knots_braids = {
    "8_2" : [1,1,1,1,1,-2,1,-2],
    "8_5" : [1,1,1,-2,1,1,1,-2],
    "8_7" : [-1,-1,-1,-1,2,-1,2,2],
    "8_9" : [1,1,1,-2,1,-2,-2,-2],
    "8_10" : [-1,-1,-1,2,-1,-1,2,2],
    "8_16" : [-1,-1,2,-1,-1,2,-1,2],
    "8_17" : [1,1,-2,1,-2,1,-2,-2],
    "8_18" : [1,-2,1,-2,1,-2,1,-2],
    "8_19" : [1,1,1,2,1,1,1,2],
    "8_20" : [1,1,1,-2,-1,-1,-1,-2],
    "8_21" : [1,1,1,2,-1,-1,2,2]
}

knot = "8_2"
braid = [2]+[sign(s)*(abs(s)+2) for s in knots_braids[knot]]+[-2]
#I.apply_braid(braid)

In [ ]:
I.make_differential(max_number_of_dots=1, method="pypardiso")

In [ ]:
I.find_cohomology(ZZ)